In [10]:
### first we need to include printing 
import shutil
import os
original_MODLE_NAME= 'SWAT_MODEL'
MODEL_NAME = 'SWAT_PFAS_MODEL'
NAME= 40500040502
BASE_PATH = '/data/MyDataBase/SWATGenXAppData/'
LEVEL = 'huc12'
original_SOURCE  = os.path.join(BASE_PATH, fr'SWATplus_by_VPUID/{LEVEL}/{NAME}/{original_MODLE_NAME}/Scenarios/Default/TxtInOut')
SOURCE = os.path.join(BASE_PATH, fr'SWATplus_by_VPUID/{LEVEL}/{NAME}/{MODEL_NAME}/Scenarios/Default/TxtInOut')
if not os.path.exists(SOURCE):
    shutil.copytree(original_SOURCE,SOURCE)

In [11]:
def revising_print(original_SOURCE , SOURCE):
    print_file = os.path.join(original_SOURCE, 'print.prt')
    shutil.copy2(print_file, SOURCE)
    print_file = os.path.join(SOURCE, 'print.prt')

    with open(print_file, 'a') as file:
        file.write('pfas    y    y    y    y\n')
revising_print(original_SOURCE , SOURCE)

In [12]:
######################################### this function read SWAT+ soil database #########################
def read_soil_db(file_path):
    soils_data = []
    try:
        with open(file_path, 'r') as file:
            eof = False

            # Read title and header
            titldum = file.readline().strip()
            header = file.readline().strip()

            while not eof:
                line = file.readline()
                if not line:
                    eof = True
                    break

                # Parse soil data
                soil_info = line.strip().split()
                soil_name, nlyr, hyd_grp , dp_tot , anion_excl, perc_crk, texture = soil_info[0], int(soil_info[1]), str(soil_info[2]), str(soil_info[3]), str(soil_info[4]), str(soil_info[5]), str(soil_info[6])
                soil_layers = []

                for _ in range(nlyr):
                    layer_line = file.readline().strip()
                    layer_info = [float(val) for val in layer_line.split()]
                    soil_layers.append(layer_info)
                

                soils_data.append({
                    'soil_name': soil_name,
                    'nlyr': nlyr,
                    'hyd_grp':hyd_grp,
                    'dp_tot':dp_tot,
                    'anion_excl': anion_excl,
                    'perc_crk' : perc_crk,
                    'texture' : texture,
                    'layers': soil_layers
                })

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return soils_data


def edit_soil_data(soil_data, new_columns):
    for soil in soil_data:
        for layer in soil['layers']:
            # Add default values for new columns or use some logic to determine their values
            for col in new_columns:
                layer.append(new_columns[col])

    return soil_data


def write_soil_db(soil_data, output_file_path):
    try:
        with open(output_file_path, 'w') as file:
            # Write a title and header with new columns
            file.write("Soils Database associated with pfas database\n")
            header = "name    nlyr    hyd_grp    dp_tot    anion_excl    perc_crk    texture    "
            header += "dp    bd    awc    soil_k    carbon    clay    silt    sand    rock    alb    usle_k    ec    caco3    ph    n    kf    d50\n"
            file.write(header)

            for soil in soil_data:
                # Write soil name and other properties
                soil_info = f"{soil['soil_name']}    {soil['nlyr']}  {soil['hyd_grp']}  {soil['dp_tot']}  {soil['anion_excl']}  {soil['perc_crk']}  {soil['texture']}\n"
                file.write(soil_info)

                # Write data for each layer
                for layer in soil['layers']:
                    layer_str = ' '.join(str(val) for val in layer)
                    file.write(f"\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t{layer_str}\n")
                    
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


file_path = os.path.join(SOURCE, 'soils.sol')
soils = read_soil_db(file_path)

default_value_n = 0.5
default_value_kf = 200
default_value_d50 = 0.01
new_columns = {'n': default_value_n, 'kf': default_value_kf, 'd50': default_value_d50}
soils_edited = edit_soil_data(soils, new_columns)

output_file_path =  os.path.join(SOURCE, 'soils_pfas.sol')
write_soil_db(soils_edited, output_file_path)

In [14]:
############################################################################## modifying the file.ico for PFAS simulation #########################################################

def modifying_filecio_pfas(LEVEL, NAME, MODEL_NAME):
    filecio = fr'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/{LEVEL}/{NAME}/{MODEL_NAME}/Scenarios/Default/TxtInOut/file.cio'
    with open(filecio, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if 'exco' in line:
                if len(line.split('exco')[1].split())!=7:
                    print('modifying exco files')
                    lines[i] = 'exco              null              null              null              null              null              null    null            \n'

            if 'hru_parm_db' in line:
                if len(line.split('hru_parm_db')[1].split())!=11:
                    print('modifying hru_parm_db files')
                    lines[i] = 'hru_parm_db       plants.plt        fertilizer.frt     tillage.til       pesticide.pes      pfas.pfas     null       null      null        urban.urb         septic.sep        snow.sno          \n'

            if 'dr' == line.split()[0]:
                if len(line.split('dr')[1].split())!=7:
                    print('modifying delivery ratio files')
                    lines[i] = 'dr              null              null              null              null              null              null    null            \n'
            if 'init' == line.split()[0]:
                if len(line.split('init')[1].split())!=13:
                    print('modifying init files')
                    lines[i] = 'init              plant.ini         soil_plant.ini    om_water.ini      pest_hru.ini      pest_water.ini              pfas_hru.ini      pfas_water.ini              null          null       null       null   null      null  \n'
    
    filecio = fr'/data/MyDataBase/SWATGenXAppData/SWATplus_by_VPUID/{LEVEL}/{NAME}/{MODEL_NAME}/Scenarios/Default/TxtInOut/file.cio'
    with open(filecio,'w') as file:
        file.writelines(lines)
        
        
        
NAME = 40500040502
MODEL_NAME = 'SWAT_PFAS_MODEL'
LEVEL = 'huc12'
modifying_filecio_pfas(LEVEL, NAME, MODEL_NAME)

        